In [ ]:
import json
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Load the JSON file
with open('C://Users//Jayashrinidhi V//OneDrive//Documents//VScode//Justice_bot//doj_website_data.json') as file:
    data = json.load(file)

# Initialize lists for training data
training_sentences = []
training_labels = []
labels = []

# Process the JSON data
for item in data:
    # Assuming each item in the list is a dictionary with 'link_text', 'url', 'page_content'
    page_content = item.get('page_content', '')
    if page_content:
        training_sentences.append(page_content)
        training_labels.append('page_content')  # Label for the content
    
    # You can also include 'link_text' and 'url' if needed
    link_text = item.get('link_text', '')
    if link_text:
        training_sentences.append(link_text)
        training_labels.append('link_text')  # Label for the link text
    
    url = item.get('url', '')
    if url:
        training_sentences.append(url)
        training_labels.append('url')  # Label for the URL

# Update labels list if using label encoding
for label in ['page_content', 'link_text', 'url']:
    if label not in labels:
        labels.append(label)

# Encode the labels
label_encoder = LabelEncoder()
training_labels = label_encoder.fit_transform(training_labels)

# Tokenize the sentences
vocab_size = 1000  # Adjust as necessary
max_length = 200  # Adjust based on your data
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(training_sentences)
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')

# Convert labels to categorical format if using a neural network
training_labels = to_categorical(training_labels, num_classes=len(labels))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, training_labels, test_size=0.2, random_state=42)

# Now you can proceed with building, training, and evaluating your model


In [1]:
###
import requests
from bs4 import BeautifulSoup
import json

# URL of the website
url = 'https://doj.gov.in/'

# Send a request to fetch the HTML content
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Extract text content from paragraphs
def extract_text(soup):
    paragraphs = [p.get_text() for p in soup.find_all('p')]
    return '\n'.join(paragraphs)

# Extract all links
def extract_links(soup):
    links = [{'text': a.get_text(), 'href': a['href']} for a in soup.find_all('a', href=True)]
    return links

# Extract headers (H1, H2, H3, etc.)
def extract_headers(soup):
    headers = [{'tag': header.name, 'text': header.get_text()} for header in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])]
    return headers

# Prepare the data
data = {
    'text_content': extract_text(soup),
    'links': extract_links(soup),
    'headers': extract_headers(soup)
}

# Convert to JSON
json_file_path = 'doj_website_data.json'
with open(json_file_path, 'w') as json_file:
    json.dump(data, json_file, indent=4)

print(f"Data has been saved to {json_file_path}")


Data has been saved to doj_website_data.json


In [2]:
#scrape from links too

import requests
from bs4 import BeautifulSoup
import json
import time

# Base URL of the website
base_url = 'https://doj.gov.in/'

# Send a request to fetch the HTML content of the main page
response = requests.get(base_url)
soup = BeautifulSoup(response.content, 'html.parser')

# Extract links from the main page
def extract_links(soup, base_url):
    links = []
    for a in soup.find_all('a', href=True):
        href = a['href']
        if href.startswith('/'):  # Handle relative URLs
            href = base_url + href[1:]
        elif not href.startswith('http'):
            href = base_url + href
        links.append({'text': a.get_text(), 'href': href})
    return links

# Extract text from a single page
def extract_text_from_page(url):
    try:
        page_response = requests.get(url)
        page_soup = BeautifulSoup(page_response.content, 'html.parser')
        paragraphs = [p.get_text() for p in page_soup.find_all('p')]
        return '\n'.join(paragraphs)
    except Exception as e:
        print(f"Failed to scrape {url}: {e}")
        return ""

# Extract data from all linked pages
def extract_data_from_links(links):
    data = []
    for link in links:
        print(f"Scraping {link['href']}")
        text_content = extract_text_from_page(link['href'])
        data.append({
            'link_text': link['text'],
            'url': link['href'],
            'page_content': text_content
        })
        time.sleep(1)  # Be polite to the server by sleeping between requests
    return data

# Get all links from the main page
links = extract_links(soup, base_url)

# Scrape data from each linked page
scraped_data = extract_data_from_links(links)

# Save the scraped data to a JSON file
json_file_path = 'doj_full_scrape.json'
with open(json_file_path, 'w') as json_file:
    json.dump(scraped_data, json_file, indent=4)

print(f"Scraped data has been saved to {json_file_path}")


Scraping https://doj.gov.in/#SkipContent
Scraping https://doj.gov.in/javascript:void(0);
Scraping https://doj.gov.in/javascript:void(0);
Scraping https://www.facebook.com/MLJ.GovIndia
Scraping https://twitter.com/MLJ_GoI
Scraping https://www.instagram.com/mlj_goi/
Scraping https://www.kooapp.com/profile/mlj_goi
Scraping https://www.youtube.com/channel/UC4dBRceizgOEHuZ4Tv1wpuQ
Scraping https://doj.gov.in/sitemap/


KeyboardInterrupt: 